In [4]:
import xgboost as xgb
import pandas as pd
import numpy as np

train_path = "../dataset/all/train.csv"
df = pd.read_csv(train_path,index_col="ID_code")

test_path = "../dataset/all/test.csv"
df_eval = pd.read_csv(test_path,index_col="ID_code")

dfLabel = df['target']
df = df.drop('target',axis=1)


msk = np.random.rand(len(df)) < 0.8
###训练集
train = df[msk]
train_label = dfLabel[msk]
###用来检测训练集的auc
train_small_msk = np.random.rand(len(train)) < 0.2
train_small = train[train_small_msk]
train_small_label = train_label[train_small_msk]
###测试集
test = df[~msk]
test_label = dfLabel[~msk]



In [5]:
#le = preprocessing.LabelEncoder()
#dfLabel = le.fit(df['target']).transform(df['label'])

from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from sklearn.metrics import auc

tree_model = XGBClassifier(
silent=1 ,#设置成1则没有运行信息输出，最好是设置为0.是否在运行升级时打印消息。
#nthread=4,# cpu 线程数 默认最大
learning_rate= 0.01, # 如同学习率
min_child_weight=1,
nthread=4,  # 并行数
# 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
#，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
#这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。
max_depth=5, # 构建树的深度，越大越容易过拟合
gamma=0.1,  # 树的叶子节点上作进一步分区所需的最小损失减少,越大越保守，一般0.1、0.2这样子，对叶节点个数的惩罚系数。
subsample=1, # 随机采样训练样本 训练实例的子采样比
max_delta_step=0,#最大增量步长，我们允许每个树的权重估计。
colsample_bytree=1, # 生成树时进行的列采样
reg_lambda=1,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
#reg_alpha=0, # L1 正则项参数
#scale_pos_weight=1, #如果取值大于0的话，在类别样本不平衡的情况下有助于快速收敛。平衡正负权重
#objective= 'multi:softmax', #多分类的问题 指定学习任务和相应的学习目标
#num_class=10, # 类别数，多分类与 multisoftmax 并用
n_estimators=100, #树的个数
seed=1000 #随机种子
#eval_metric= 'auc'
)

tree_model.fit(train,train_label )


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0.1, learning_rate=0.01, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1000, silent=1,
       subsample=1)

In [23]:
def eval_res(test_label,preds):
	y = test_label
	scores = preds
	y_pred = (scores >= 0.5)*1
	print('AUC: %.4f' % metrics.roc_auc_score(y,scores))
	print('ACC: %.4f' % metrics.accuracy_score(y,y_pred))
	print('Recall: %.4f' % metrics.recall_score(y,y_pred))
	print('F1-score: %.4f' %metrics.f1_score(y,y_pred))
	print('Precesion: %.4f' %metrics.precision_score(y,y_pred))
    
preds = tree_model.predict_proba(test)   
eval_res(test_label,preds[:,1])


AUC: 0.7172
ACC: 0.8989
Recall: 0.0005
F1-score: 0.0010
Precesion: 1.0000


In [25]:
train_oh

array([[58, 58, 57, ..., 56, 61, 49],
       [58, 58, 57, ..., 56, 61, 49],
       [36, 35, 44, ..., 56, 45, 49],
       ...,
       [51, 51, 57, ..., 42, 61, 58],
       [58, 58, 57, ..., 41, 60, 49],
       [51, 51, 57, ..., 41, 61, 49]], dtype=int32)

In [26]:
train_oh = tree_model.apply(train)
tm_enc = OneHotEncoder(categories='auto',sparse = 'false')
res_date = tm_enc.fit(train_oh).transform(train_oh)
all_cat = tm_enc.categories_

In [27]:
#  y field_1:index_1:value_1 field_2:index_2:value_2   ...
### 转换train 数据集合 
all_cat = tm_enc.categories_

def convert2ffm(lines):
    str_res=""
    for item_ind in range(len(lines)):
        cats = all_cat[item_ind]
        try:
            index = cats.tolist().index(lines[item_ind])+1
        except:
            index = 0
        str_one= " " + str(item_ind) + ":" + str(index) + ":1"
        str_res+=str_one
    return str_res  

count=0
with open("train_ffm.libffm","w") as wrop:
    for label , lines in zip(train_label,train_oh):  
        if count%1000 == 0:
            print count
        res_format = str(label) + convert2ffm(lines) + "\n"
        wrop.write(res_format) 
        count = count+1
wrop.close()  

        

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [28]:
test_oh = tree_model.apply(test)
test_date = tm_enc.transform(test_oh)

In [29]:
### 转换test数据集合 
all_cat = tm_enc.categories_
def convert2ffm(lines):
    str_res=""
    for item_ind in range(len(lines)):
        cats = all_cat[item_ind]
        try:
            index = cats.tolist().index(lines[item_ind])+1
        except:
            index = 0
        str_one= " " + str(item_ind) + ":" + str(index) + ":1"
        str_res+=str_one
    return str_res  

count=0
with open("test_ffm.libffm","w") as wrop:
    for label , lines in zip(test_label,test_oh):  
        if count%1000 == 0:
            print count
        res_format = str(label) + convert2ffm(lines) + "\n"
        wrop.write(res_format)
        count = count+1
wrop.close() 

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000


In [30]:
import numpy as np
###### 如果加入dense vector会强烈降低auc
# train_add = np.hstack((train.values,res_date.toarray()))
# test_add = np.hstack((test.values,test_date.toarray()))

In [31]:
import xlearn as xl

ffm_model1 = xl.create_ffm() 
ffm_model1.setTrain("train_ffm.libffm") 
ffm_model1.setValidate("test_ffm.libffm")

ffm_model2 = xl.create_ffm() 
ffm_model2.setTrain("train_ffm.libffm") 
ffm_model2.setValidate("test_ffm.libffm")

ffm_model3 = xl.create_ffm() 
ffm_model3.setTrain("train_ffm.libffm") 
ffm_model3.setValidate("test_ffm.libffm")

In [36]:
import xlearn as xl
ffm_model1 = xl.create_ffm() 
ffm_model1.setTrain("train_ffm.libffm") 
ffm_model1.setValidate("test_ffm.libffm")

param1 = {'task':'binary', 'lr':0.1, 'lambda':0.002,'epoch':20,'opt':'ftrl'}
ffm_model1.fit(param1, "model1.out")

ffm_model1.setSigmoid()
ffm_model1.setTest("test_ffm.libffm")
ffm_model1.predict("model1.out", "output1.txt")
preds = numpy.loadtxt('output1.txt')
eval_res(test_label,preds)

AUC: 0.7452
ACC: 0.9010
Recall: 0.0686
F1-score: 0.1230
Precesion: 0.5915


In [33]:
import numpy

def eval_res(test_label,preds):
	y = test_label
	scores = preds
	y_pred = (scores >= 0.5)*1
	print('AUC: %.4f' % metrics.roc_auc_score(y,scores))
	print('ACC: %.4f' % metrics.accuracy_score(y,y_pred))
	print('Recall: %.4f' % metrics.recall_score(y,y_pred))
	print('F1-score: %.4f' %metrics.f1_score(y,y_pred))
	print('Precesion: %.4f' %metrics.precision_score(y,y_pred))
    
ffm_model1.setSigmoid()
ffm_model1.setTest("test_ffm.libffm")
ffm_model1.predict("model1.out", "output1.txt")
preds = numpy.loadtxt('output1.txt')
eval_res(test_label,preds)

AUC: 0.7439
ACC: 0.9004
Recall: 0.0716
F1-score: 0.1269
Precesion: 0.5598


In [ ]:
param1 = {'task':'binary', 'lr':0.1, 'lambda':0.002,'epoch':20,'k':8,'opt':'sgd'}
param2 = {'task':'binary', 'lr':0.1, 'lambda':0.002,'epoch':20,'k':8,'opt':'adagrad'}
param3 = {'task':'binary', 'lr':0.1, 'lambda':0.002,'epoch':20,'k':8,'opt':'ftrl'}


# Train model
ffm_model1.fit(param1, "model1/model1.out")
ffm_model2.fit(param2, "model2/model2.out")
ffm_model3.fit(param3, "model3/model3.out")


In [ ]:
# ffm_model.setSign()
# ffm_model.setTest("bb.libsvm")
# ffm_model.predict("model.out", "output_label.txt")

ffm_model1.setSigmoid()
ffm_model1.setTest("test_ffm.libffm")
ffm_model1.predict("model1/model1.out", "model1/output1.txt")

ffm_model2.setSigmoid()
ffm_model2.setTest("test_ffm.libffm")
ffm_model2.predict("model2/model2.out", "model2/output2.txt")

ffm_model3.setSigmoid()
ffm_model3.setTest("test_ffm.libffm")
ffm_model3.predict("model3/model3.out", "model3/output3.txt")



In [ ]:
import numpy
preds = numpy.loadtxt('model1/output1.txt')
eval_res(test_label,preds)

In [54]:
import numpy
preds = numpy.loadtxt('model2/output2.txt')
eval_res(test_label,preds)

AUC: 0.7474
ACC: 0.9041
Recall: 0.0782
F1-score: 0.1386
Precesion: 0.6063


In [55]:
import numpy
preds = numpy.loadtxt('model3/output3.txt')
eval_res(test_label,preds)

AUC: 0.7521
ACC: 0.9037
Recall: 0.0879
F1-score: 0.1525
Precesion: 0.5748
